In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_csv(r"C:\Users\Pushpa Kumari\Desktop\PYTHON\forestfires.csv")
df.head(10)

list1=[0 for i in range(517)]
count=0
for index,rows in df.iterrows():
    if rows["area"]==0:
        list1[index]=0
    else:
        list1[index]=1
area_b=pd.DataFrame(list1)
df=pd.concat([df,area_b],axis=1)
df.rename({0:"area_b"},axis=1,inplace=True)
df.drop(["area"],axis=1,inplace=True)

from sklearn.preprocessing import LabelEncoder
le_Gender=LabelEncoder()
#adding the le_gender column to the input var df(x)
df["month_b"]=le_Gender.fit_transform(df["month"])
df["day_b"]=le_Gender.fit_transform(df["day"])
df.drop(["month","day"],axis=1,inplace=True)

In [2]:
count=0
for index,row in df.iterrows():
    if (row["Y"]>6) or (row["Y"]<3) :
        df=df.drop(index)
        count+=1
print(df["Y"].count())
#plt.boxplot(df["Y"])
#df.info()
print(count)

466
51


In [3]:
count=0
for index,row in df.iterrows():
    if (row["FFMC"]>96) or (row["FFMC"]<88) :
        df=df.drop(index)
        count+=1
print(df["FFMC"].count())
#plt.boxplot(df["FFMC"])
#df.info()
print(count)

393
73


In [4]:
#removing outleirs of DMC
count=0
for index,row in df.iterrows():
    if (row["DMC"]>215) or (row["DMC"]<5) :
        df=df.drop(index)
        count+=1
print(df["DMC"].count())
#plt.boxplot(df["DMC"])
#df.info()
print(count)

355
38


In [5]:
#removing outleirs of DC
count=0
for index,row in df.iterrows():
    if (row["DC"]>800) or (row["DC"]<450) :
        df=df.drop(index)
        count+=1
print(df["DC"].count())
#plt.boxplot(df["DC"])
#df.info()
print(count)

275
80


In [6]:
#removing outleirs of ISI
count=0
for index,row in df.iterrows():
    if (row["ISI"]>15.5) or (row["ISI"]<0) :
        df=df.drop(index)
        count+=1
print(df["ISI"].count())
#plt.boxplot(df["ISI"])
#df.info()
print(count)

247
28


In [7]:
#removing outleirs of temp
count=0
for index,row in df.iterrows():
    if (row["temp"]>30) or (row["temp"]<12) :
        df=df.drop(index)
        count+=1
print(df["temp"].count())
#plt.boxplot(df["temp"])
#df.info()
print(count)

230
17


In [8]:
df.drop(["rain"],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 1 to 515
Data columns (total 12 columns):
X          230 non-null int64
Y          230 non-null int64
FFMC       230 non-null float64
DMC        230 non-null float64
DC         230 non-null float64
ISI        230 non-null float64
temp       230 non-null float64
RH         230 non-null int64
wind       230 non-null float64
area_b     230 non-null int64
month_b    230 non-null int32
day_b      230 non-null int32
dtypes: float64(6), int32(2), int64(4)
memory usage: 21.6 KB


In [9]:
for column in ["X","Y","FFMC","DMC","DC","ISI","temp","RH","wind","month_b","day_b"]:
    min_=float(df[column].min())
    max_=float(df[column].max())
    for index,row in df.iterrows():
        df[column]=df[column].replace(row[column],(row[column]-min_)/(max_-min_))


In [10]:
df.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,area_b,month_b,day_b
1,0.750,0.333333,0.320513,0.000000,0.634309,0.184466,0.325581,0.305085,0.058824,0,0.9,0.833333
2,0.750,0.333333,0.320513,0.048368,0.685888,0.184466,0.127907,0.305085,0.105882,0,0.9,0.333333
5,0.875,1.000000,0.538462,0.290793,0.109533,0.961165,0.569767,0.237288,0.588235,0,0.0,0.500000
6,0.875,1.000000,0.538462,0.311772,0.131556,0.359223,0.680233,0.203390,0.317647,0,0.0,0.166667
8,0.875,1.000000,0.371795,0.548368,0.702405,0.213592,0.040698,0.813559,0.588235,0,0.0,0.833333


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x=df.drop(["area_b","month_b"],axis=1) #month_b has so many zeroes after normalization, hence dropping it from x
y=df["area_b"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=100)
logmodel=LogisticRegression()
logmodel.fit(x_train,y_train)

y_pred=logmodel.predict(x_test)

C:\Users\Pushpa Kumari\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
bestfeatures=SelectKBest(score_func=f_classif,k=7)
fit=bestfeatures.fit(x,y)
dfscores=pd.DataFrame(fit.scores_)
dfcolumns=pd.DataFrame(x)

In [13]:
dfscores

,0
0,2.237210
1,1.009861
2,4.920123
3,7.168572
4,0.521769
5,4.098765
6,3.082386
7,0.110887
8,0.016892
9,0.632231


In [14]:
from sklearn.model_selection import train_test_split
#x=df.drop(["DMC","FFMC","ISI","temp"],axis=1)
x=x.loc[ : ,["DMC","FFMC","ISI","temp","X","Y","day_b"]]
y=df["area_b"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

from sklearn.linear_model import LogisticRegression
logmodel=LogisticRegression()
logmodel.fit(x_train,y_train)

C:\Users\Pushpa Kumari\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
predictions=logmodel.predict(x_test)
#predictions
print(y_test)
print(predictions[0])

107    0
252    1
357    0
308    0
360    1
288    0
210    1
172    1
441    1
272    1
385    1
46     0
197    1
142    1
361    1
137    0
170    1
368    1
244    1
185    1
55     0
6      0
154    1
145    1
54     0
403    0
262    1
364    1
341    0
321    1
253    1
381    1
310    0
109    0
234    1
179    1
232    1
95     0
263    1
307    1
34     0
493    1
52     0
99     0
482    1
251    1
Name: area_b, dtype: int64
1


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.5652173913043478